# Deploy a Python model in Machine Learning Server with `azureml-model-management-sdk` Package 

                               **Applied to: Machine Learning Server 9.2**

## 1. Read in the mtcars dataset

From your local machine, let's begin by reading in the data we will use to build our model. We will use the dataset `mtcars`. 

In [2]:
# -- Import the dataset from the microsoftml package
from microsoftml.datasets.datasets import get_dataset
mtcars = get_dataset('mtcars')

# -- Represent the dataset as a dataframe.
mtcars = mtcars.as_df()

# -- print top rows of data to inspect the data
mtcars.head()

,car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


## 2. Authenticate and initiate the  `DeployClient`

There are several ways to authentication against Machine Learning Server from your local machine. The method you choose should match the authentication defined by your administrator. Please contact your administrator for authentication credentials. 

Learn more about authenticating in Python with Active Directory LDAP or Azure Active Directory in the article "[Connecting to Machine Learning Server in Python](https://docs.microsoft.com/en-us/r-server/operationalize/python/how-to-authenticate-in-python )."

For simplicity, this example uses the local 'admin' account for authentication.  

1. Begin by importing the DeployClient and MLServer classes from the [azureml-model-management-sdk package](https://docs.microsoft.com/en-us/r-server/python-reference/azureml-model-management-sdk/azureml-model-management-sdk) so you can connect to Machine Learning Server (`use=MLServer`).

1. Then, fill in your own connection details for the host and context into the corresponding fields. Learn more in the article "[Connecting to Machine Learning Server in Python](https://docs.microsoft.com/en-us/r-server/operationalize/python/how-to-authenticate-in-python )."

In [3]:
# -- Import the DeployClient and MLServer classes from the azureml-model-management-sdk package.
from azureml.deploy import DeployClient
from azureml.deploy.server import MLServer

# -- Define the location of the ML Server --
# -- for local onebox for Machine Learning Server: http://localhost:12800
HOST = 'http://localhost:12800'
context = ('admin', 'Happiness_1')
client = DeployClient(HOST, use=MLServer, auth=context)

You are now authenticated. 

You have the **DeployClient**, which is the handle through which you can interact with the web service management APIs to deploy, list, consume and so on. 

## 3. Create and run a linear model locally

Now that you have built the authentication logic into your application, you can interact with the core APIs using functions in azureml-model-management-sdk to create a model and publish it as a web service.

Create a GLM model called `cars_model` using the dataset `mtcars` we imported before. Using horsepower (hp) and weight (wt), this model estimates the probability that a vehicle has been fitted with a manual transmission.

We use the [rx_lin_mod](https://docs.microsoft.com/en-us/r-server/python-reference/revoscalepy/rx-lin-mod) function from the [revoscalepy package](https://docs.microsoft.com/en-us/r-server/python-reference/revoscalepy/revoscalepy-package) to build the model.

In [4]:
# -- import the needed classes and functions
import pandas as pd
from revoscalepy import rx_lin_mod, rx_predict

# -- using rx_lin_mod from revoscalepy package
# -- create glm model with `mtcars` dataset
cars_model = rx_lin_mod(
    formula='am ~ hp + wt',
    data=mtcars)

# -- provide some sample inputs to test the model
mydata = pd.DataFrame({
    'hp':[120],
    'wt':[2.8]
})
mydata

# -- predict locally
rx_predict(cars_model, data=mydata)

Rows Read: 32, Total Rows Processed: 32, Total Chunk Time: Less than .001 seconds 
Computation time: 0.038 seconds.
Rows Read: 1, Total Rows Processed: 1, Total Chunk Time: Less than .001 seconds 


,am_Pred
0,0.533267


Examine the results of the locally executed code. You can compare these results to the results of the web service in this next step.

## 4. Publish the model as a web service

Now let's:
+ Define an initialization function 
+ Produce a prediction function
+ Publish the linear model as a Python web service

Define an `init` function that we can bootstrap to the web service. This init function will handle service initialization. This is a great way to improve performance by loading the needed packages, datasets, and global variables when the service is called the first time. Then, the server won't waste time reloading these each time it is run. 

In [5]:
# --Define an `init` function to handle service initialization --
def init():
    import pandas as pd
    from revoscalepy import rx_predict
    from matplotlib import pyplot as plt

Produce a prediction function called `manualTransmission` that can use the `cars_model` model. This function will be part of the code `code_fn` supplied when the service is published.

In [6]:
def manualTransmission(hp, wt):
    import pandas as pd
    from revoscalepy import rx_predict
    from matplotlib import pyplot as plt
    
    # -- make the prediction use model `cars_model` and input data --
    newData = pd.DataFrame({'hp':[hp], 'wt':[wt]})
    answer = rx_predict(cars_model, newData, type='response')
    
    # -- save some files to demonstrate the ability to return file artifacts --
    answer.to_csv('answer.csv')
    x = [1,2,3]
    plt.scatter(x, x)
    plt.savefig('image.png')
    
    # return prediction
    return answer

Now we can publish the model as a web service called `"cars_model"` to Machine Learning Server. This service uses the model `carsModel`, the function `manualTransmission`, and the `init` function. As an input, the service takes a list of vehicle horsepower and vehicle weight represented as a float. As an output, a percentage as a dataframe for the probability each vehicle has of being fitted with a manual transmission.

When publishing a service, you can specify its name and version, the code functions, the inputs, and the outputs needed for application integration as well as other parameters. 

NOTE ON THE BASICS OF SERVICE PUBLISHING API:

The fluent APIS are designed for optional configurations where the readability of the invocation is close to that of the ordinary written prose (grammatical structure). A publish can be initiated by invoking the `client.service(name)` object then calling `.deploy()`  to send the publish request.

In [7]:
service_name = 'TxService'

service = client.service(service_name)\
        .version('1.0')\
        .code_fn(manualTransmission, init)\
        .inputs(hp=float, wt=float)\
        .outputs(answer=pd.DataFrame)\
        .models(cars_model=cars_model)\
        .description('My first python model')\
        .artifacts(['answer.csv', 'image.png'])\
        .deploy()

## 5. Explore the published web service

Now let's:
+ Use the help function to explore the published service

+ Print the capabilities that define the service holdings: service name, version, descriptions, inputs, outputs, and the name of the function to be consumed

Call the `help` function on any `azureml-model-management-sdk` functions, even those we dynamically generated ones to learn more about them.

In [8]:
print(help(service))

Help on TxserviceService in module azureml.deploy.server.service object:

class TxserviceService(Service)
 |  Service object from metadata.
 |  
 |  Method resolution order:
 |      TxserviceService
 |      Service
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, service, http_client)
 |      Constructor
 |      
 |      :param service:
 |      :param http_client:
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  batch(self, records, parallel_count=10)
 |      Register a set of input records for batch execution on this service.
 |      
 |      :param records: The `data.frame` or `list` of
 |             input records to execute.
 |      :param parallel_count: Number of threads used to process entries in
 |             the batch. Default value is 10. Please make sure not to use too
 |             high of a number because it might negatively impact performance.
 |      :return: The `Batch` object to control service batching
 |              lifecycle

Explore all available functions on the service object by calling `capabilities`.

In [9]:
service.capabilities()

{'api': '/api/TxService/1.0',
 'artifacts': ['answer.csv', 'image.png'],
 'creation_time': '2017-09-14T21:06:29.7309542',
 'description': 'My first python model',
 'inputs': [{'name': 'hp', 'type': 'numeric'},
  {'name': 'wt', 'type': 'numeric'}],
 'inputs_encoded': [{'name': 'hp', 'type': 'float'},
  {'name': 'wt', 'type': 'float'}],
 'name': 'TxService',
 'operation_id': 'manualTransmission',
 'outputs': [{'name': 'answer', 'type': 'data.frame'}],
 'outputs_encoded': [{'name': 'answer', 'type': 'pandas.DataFrame'}],
 'public-functions': {'batch': 'batch(records, parallel_count=10)',
  'capabilities': 'capabilities()',
  'get_batch': 'get_batch(execution_id)',
  'list_batch_execution': 'list_batch_execution()',
  'manualTransmission': 'manualTransmission(self,hp,wt)',
  'swagger': 'swagger(json=True)'},
 'published_by': 'admin',
 'runtime': 'Python',
 'snapshot_id': '15225aef-4c0b-4c60-94a3-382ad60955c5',
 'swagger': 'http://localhost:12800/api/TxService/1.0/swagger.json',
 'version':

## 6. Consume the service and make a prediction

Once the service is published, you can consume it using the _service api_ object returned from `.deploy()` to verify that the results are as expected. The results should match the results obtained when the model was run locally earlier. 

In [10]:
res = service.manualTransmission(120, 2.8)

# -- Print Response object to inspect what was returned --
# disabled as it will print long binary string of the returned 
# image artifact
# print(res)

# -- Pluck out the named output `answer` as defined during publishing and print --
print(res.output('answer'))

# -- You can also explore all available functions on the service object by calling `service.capabilities()`

    am_Pred
0  0.533267


Anyone can test and consume the service using its auto-generated Swagger-based JSON file. This Swagger-based JSON file is specific to a given version of a service. You can easily get this file during the same authenticated session in which you published the service. It can be downloaded to the local file system.  **You can share the resulting file with application developers or others testing your service.**

In [11]:
# -- Retrieve the URL of the swagger file for this service.
cap = service.capabilities()
swagger_URL = cap['swagger']
print(swagger_URL)

http://localhost:12800/api/TxService/1.0/swagger.json


In [12]:
# -- Print the contents of the swagger doc
print(service.swagger())


{
  "swagger": "2.0",
  "info": {
    "description": "My first python model",
    "title": "TxService",
    "version": "1.0"
  },
  "schemes": [
    "http",
    "https"
  ],
  "securityDefinitions": {
    "Bearer": {
      "type": "apiKey",
      "name": "Authorization",
      "in": "header"
    }
  },
  "tags": [
    {
      "name": "TxService",
      "description": "My first python model"
    }
  ],
  "consumes": [
    "application/json"
  ],
  "produces": [
    "application/json"
  ],
  "paths": {
    "/login": {
      "post": {
        "operationId": "login",
        "summary": "Logs the user in",
        "parameters": [
          {
            "name": "loginRequest",
            "in": "body",
            "required": true,
            "schema": {
              "$ref": "#/definitions/LoginRequest"
            }
          }
        ],
        "tags": [
          "User"
        ],
        "responses": {
          "200": {
            "description": "Login successful",
            "sch

You could also call 'help' function on any `azureml-model-management-sdk` functions, even those we dynamically generated ones.

## 7. Manage services on the server

### List services

To see the current list of services available on the Machine Learning Server, call `list_services` on the `DeployClient` object we initiated at the beginning of the notebook.

If you specify:
1. Nothing other that list_services, the list returned contains all services on the running server including those created by other users or in different languages.
    
2. A service name, the list returned contains all versions of this named service.
    
3. A service name and a version, the list returned contains details only for this specific service.

In [13]:
# -- List all services --
client.list_services()

[{'code': "def manualTransmission(hp, wt):\n    import pandas as pd\n    from revoscalepy import rx_predict\n    from matplotlib import pyplot as plt\n    \n    # -- make the prediction use model `cars_model` and input data --\n    newData = pd.DataFrame({'hp':[hp], 'wt':[wt]})\n    answer = rx_predict(cars_model, newData, type='response')\n    \n    # -- save some files to demonstrate the ability to return file artifacts --\n    answer.to_csv('answer.csv')\n    x = [1,2,3]\n    plt.scatter(x, x)\n    plt.savefig('image.png')\n    \n    # return prediction\n    return answer\n\nanswer = manualTransmission(hp, wt)",
  'creationTime': '2017-09-14T21:06:29.7309542',
  'description': 'My first python model',
  'initCode': 'def init():\n    import pandas as pd\n    from revoscalepy import rx_predict\n    from matplotlib import pyplot as plt\n\ninit()',
  'inputParameterDefinitions': [{'name': 'hp', 'type': 'numeric'},
   {'name': 'wt', 'type': 'numeric'}],
  'myPermissionOnService': 'read/w

In [14]:
# -- List all services by this name --
client.list_services('TxService')

[{'code': "def manualTransmission(hp, wt):\n    import pandas as pd\n    from revoscalepy import rx_predict\n    from matplotlib import pyplot as plt\n    \n    # -- make the prediction use model `cars_model` and input data --\n    newData = pd.DataFrame({'hp':[hp], 'wt':[wt]})\n    answer = rx_predict(cars_model, newData, type='response')\n    \n    # -- save some files to demonstrate the ability to return file artifacts --\n    answer.to_csv('answer.csv')\n    x = [1,2,3]\n    plt.scatter(x, x)\n    plt.savefig('image.png')\n    \n    # return prediction\n    return answer\n\nanswer = manualTransmission(hp, wt)",
  'creationTime': '2017-09-14T21:06:29.7309542',
  'description': 'My first python model',
  'initCode': 'def init():\n    import pandas as pd\n    from revoscalepy import rx_predict\n    from matplotlib import pyplot as plt\n\ninit()',
  'inputParameterDefinitions': [{'name': 'hp', 'type': 'numeric'},
   {'name': 'wt', 'type': 'numeric'}],
  'myPermissionOnService': 'read/w

### Get service objects

You could also retrieve a web service object for consumption by calling `get_service` on the `DeployClient` object. 

In [15]:
# -- Return the web service object with this name and version --
svc = client.get_service('TxService', version='1.0')
print(svc)

<TxserviceService> 
{   '_api': '/api/TxService/1.0',
    '_http_client': <azureml.common.http_client.HttpClient object at 0x0000004D1F79D898>,
    '_service': {   '_fn': {   'args': 'self,hp,wt',
                               'name': 'manualTransmission'},
                    'code': 'def '
                            'manualTransmission(hp, '
                            'wt):\n'
                            '    '
                            'import '
                            'pandas '
                            'as '
                            'pd\n'
                            '    '
                            'from '
                            'revoscalepy '
                            'import '
                            'rx_predict\n'
                            '    '
                            'from '
                            'matplotlib '
                            'import '
                            'pyplot '
                            'as '
                 

### Delete services

You can call `delete_service` on the `DeployClient` object to delete a specific service on the running Machine Learning Server.

In [16]:
client.delete_service('TxService', version='1.0')

True